In [1]:
!pip -q install langchain openai tiktoken chromadb


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip show langchain

Name: langchain
Version: 0.1.0
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\python\lib\site-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-experimental, langserve, trulens-eval


In [3]:
# !wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
# !unzip -q new_articles.zip -d new_articles

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# Replace 'PyPDFLoader' with the appropriate loader class if needed

# Specify the directory containing PDF files
directory_path = './data'

# Create a DirectoryLoader instance
loader = DirectoryLoader(directory_path, glob="./*.pdf", loader_cls=PyPDFLoader)

# Load documents from the specified directory
documents = loader.load()


In [5]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [6]:
texts

[Document(page_content='Assessment of eligibility for \nconcessionary travel passes \nfor disabled people -  \nMethod Statement', metadata={'source': 'data\\Method-statement.pdf', 'page': 0}),
 Document(page_content='West Midlands Combined \nAuthority Method Statement\nAssessment of eligibility for concessionary \ntravel passes for disabled people \n1. Purpose of document\n1.1 \nThis document sets out West Midlands \nCombined Authority practice when \nconsidering applications for concessionary \ntravel passes for disabled people. \n1.2\nThe statement describes a detailed \ninterpretation of the seven eligibility \n‘categories’ defined in the Transport Act \n2000. West Midlands Combined Authority \nwill apply this statement in order to arrive \nat a decision on eligibility for a disabled \nresident.\n1.3\nThis statement does not describe the \napplication process itself.\n1.4 \nIf you would like a copy of this statement in \na form more suited to your needs, please call \n0345 303 6760.

In [7]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
persist_directory = 'db'

embedding=OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=texts,embedding=embedding,persist_directory=persist_directory)

c:\python\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:

from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

c:\python\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
question = "What can call off hawkeye drill"
'''docs = vectorstore.similarity_search(question)
len(docs)'''

'docs = vectorstore.similarity_search(question)\nlen(docs)'

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),)
result = qa_chain({"query": question})

c:\python\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [11]:
#This is where you stopped

print(result)

{'query': 'What can call off hawkeye drill', 'result': 'Based on the given context, there is no information about a "hawkeye drill" or any specific event or activity that can call it off. Therefore, I don\'t have enough information to answer your question.'}


In [12]:
#To Retrieve The source documents from which the informations were gotten

# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=vectorstore.as_retriever(),
                                  return_source_documents=True)


## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [13]:
# full example
query = "What can call off hawkeye drill?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Based on the provided context, there is no information about a "hawkeye drill" or what could call it off. Therefore, I don't have enough information to answer your question.


Sources:
data\Method-statement.pdf
data\Method-statement.pdf
data\Method-statement.pdf
data\Method-statement.pdf


In [14]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

NameError: name 'vectordb' is not defined